# Simulated spectra for signals and noise

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 1

import numpy as np
import matplotlib
matplotlib.rcParams['figure.dpi'] = 90

import matplotlib.pyplot as plt

from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('png','pdf')

%aimport detectors

diodes = detectors.diodes
TIA = detectors.TIA()

savefig = True
ifig = 1

AP PDB-C164 disc. PIN
  QE = 81.3 %
  Resp. = 0.43 A/W
  Dark current (total or vol.) = 1.0 nA
  Junction cap = 7.0 pF
  Shunt res. = 500 MOhms
  Ref. bias voltage = 10 V
  Ref. temp. = 20 C
Hama. S13620 8x8 PIN
  QE = 92.6 %
  Resp. = 0.49 A/W
  Dark current (total or vol.) = 0.3 nA
  Junction cap = 15.0 pF
  Shunt res. = 250 MOhms
  Ref. bias voltage = 9 V
  Ref. temp. = 20 C
AP SD197 disc. APD
  QE = 76.0 %
  Resp. = 0.40 A/W
  Dark current (total or vol.) = 6.0 nA
  Junction cap = 25.0 pF
  Shunt res. = 250 MOhms
  Ref. bias voltage = 1800 V
  Ref. temp. = 20 C
  Gain = 300
  Noise factor = 2.2
Hama. S8550 4x8 APD
  QE = 85.0 %
  Resp. = 0.45 A/W
  Dark current (total or vol.) = 1.0 nA
  Junction cap = 9.0 pF
  Shunt res. = 250 MOhms
  Ref. bias voltage = 330 V
  Ref. temp. = 20 C
  Gain = 50
  Noise index = 0.20
  Noise factor = 2.2
